# RAG

A continuación se muestra una descripción general de alto nivel del sistema que queremos construir:


<img src='images/img_1.png' width="800">

# PARTE I

Empecemos cargando las variables de entorno que necesitamos utilizar.

## Setting up the model
Definamos el modelo LLM que utilizaremos como parte del flujo de trabajo.

Probamos el modelo haciendo una pregunta sencilla

El resultado del modelo es una instancia de `AIMessage` que contiene la respuesta. Podemos extraer esta respuesta encadenando el modelo con un analizador de salida [outputParser](https://python.langchain.com/docs/modules/model_io/output_parsers/).

Así es como se ve el encadenamiento del modelo con un analizador de salida:

<img src='images/chain1.png' width="1200">

Para este ejemplo, utilizaremos un `StrOutputParser` simple para extraer la respuesta como una cadena.

## Presentamos las plantillas de preguntas

Queremos contextualizar el modelo y la pregunta. [Prompt templates](https://python.langchain.com/docs/modules/model_io/prompts/quick_start) Son una forma sencilla de definir y reutilizar indicaciones.

Ahora podemos encadenar el mensaje con el modelo y el analizador de salida.

<img src='images/chain2.png' width="1200">

## Combinación de cadenas

Podemos combinar diferentes cadenas para crear flujos de trabajo más complejos. Por ejemplo, creemos una segunda cadena que traduzca la respuesta de la primera a otro idioma.

Comencemos creando una nueva plantilla de solicitud para la cadena de traducción:

Ahora podemos crear una nueva cadena de traducción que combine el resultado de la primera cadena con la solicitud de traducción.

Así es como se ve el nuevo flujo de trabajo:

<img src='images/chain3.png' width="1200">

# PARTE II

## Transcripcion de video de YouTube

El contexto que queremos enviar al modelo proviene de un video de YouTube. Descargamos el video y transcribámoslo con [OpenAI's Whisper](https://openai.com/research/whisper).

Vamos a leer la transcripción y mostrar los primeros caracteres para asegurarnos de que todo funciona como se espera.

## Usando la transcripción completa como contexto

Si intentamos invocar la cadena usando la transcripción como contexto, el modelo devolverá un error porque el contexto es demasiado largo.

Los modelos de lenguaje grandes admiten tamaños de contexto limitados. El vídeo que estamos usando es demasiado largo para que el modelo lo pueda procesar, por lo que necesitamos buscar una solución diferente.

## División de la transcripción

Dado que no podemos usar la transcripción completa como contexto para el modelo, una posible solución es dividir la transcripción en fragmentos más pequeños. Así, podemos invocar el modelo utilizando solo los fragmentos relevantes para responder a una pregunta específica:

<img src='images/system2.png' width="1200">

Comencemos cargando la transcripción en la memoria:

Hay muchas maneras de dividir un documento. En este ejemplo, usaremos un divisor simple que divide el documento en fragmentos de tamaño fijo. Consulta [Divisores de texto](https://python.langchain.com/docs/modules/data_connection/document_transformers/) para obtener más información sobre los diferentes enfoques para dividir documentos.

A modo de ejemplo, dividiremos la transcripción en fragmentos de 100 caracteres con una superposición de 20 caracteres y mostraremos los primeros fragmentos:

Para nuestra aplicación específica, utilizaremos 1000 caracteres en su lugar:

# PARTE III

## Configuración de un Vector Store

Necesitamos una forma eficiente de almacenar fragmentos de documentos, sus Embeddings y realizar búsquedas de similitud a gran escala. Para ello, usaremos un Vector Store.

Un Vector Store es una base de datos de Embeddings especializada en búsquedas rápidas de similitud.


<img src='images/chain4.png' width="1200">

Necesitamos configurar un retriever (https://python.langchain.com/docs/how_to/#retrievers). Este retriever realizará una búsqueda de similitud en el almacén vectorial y devolverá los documentos más similares al siguiente paso de la cadena.

## Configurar Pinecone

Para este ejemplo, usaremos [Pinecone](https://www.pinecone.io/).

<img src="images/pinecone.png" width="800">

El primer paso es crear una cuenta de Pinecone, configurar un índice, obtener una clave API y configurarla como variable de entorno `PINECONE_API_KEY`.

Ahora ejecutemos una búsqueda de similitud en pinecone para asegurarnos de que todo funciona:

Configuremos la nueva cadena usando Pinecone como almacén vectorial: